### Convolutional Neural Network
Using EfficientNetV2

#### Imports

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

#### Load CIFAR-10 data

In [3]:
(x_train_val, y_train_val), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


#### Normalize images

In [4]:
x_train_val = x_train_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

#### Training and validation split

In [5]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train_val, y_train_val, test_size=0.2, random_state=1234, stratify=y_train_val
)

#### One-hot encoding for labels

In [6]:
y_train_one_hot = to_categorical(y_train, 10)
y_val_one_hot = to_categorical(y_val, 10)
y_test_one_hot = to_categorical(y_test, 10)

#### Build EfficientNetV2 model

In [11]:
base_model = EfficientNetV2B0(
    include_top=False,
    input_shape=(32, 32, 3),
    weights='imagenet',
    pooling='avg'
)
base_model.trainable = True  # Fine-tune the base model

model = models.Sequential([
    base_model,
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax', kernel_regularizer=regularizers.l2(0.001))
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#### Train model

In [ ]:
history = model.fit(
    x_train, y_train_one_hot,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val_one_hot)
)

#### Test model

In [ ]:
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = y_test.flatten()

#### Metrics
Accuracy, F1 score, and confusion matrix

In [ ]:
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
cm = confusion_matrix(y_true, y_pred)

print(f"Test Accuracy: {acc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
plt.figure(figsize=(8,6))
plt.imshow(cm, cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()
plt.show()